## Conctere Model
- $min \ 2x_1 + 3x_2$
- S.T.
    - $3x_1 + 4x_2 \geq 1$
    - $x_1, x_2 \geq 0$

In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [2]:
# Defined the problem here.
model = pyo.ConcreteModel()
model.x = pyo.Var([1, 2], domain=pyo.NonNegativeReals)
model.objective = pyo.Objective(expr=2*model.x[1] + 3*model.x[2], sense=pyo.minimize)
model.constraint1 = pyo.Constraint(expr=3*model.x[1] + 4*model.x[2] >= 1)

In [3]:
# Solve the problem here.
opt = SolverFactory("glpk")
print(opt.solve(model))


Problem: 
- Name: unknown
  Lower bound: 0.666666666666667
  Upper bound: 0.666666666666667
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 2
  Number of nonzeros: 2
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.004822254180908203
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [4]:
pyo.value(model.objective)

0.666666666666666

## Part 1
- $min \ \sum_{i=1}^4 x_i$

In [5]:
# Check binary(max and min) | check nonnegative | check nonpositive
model = pyo.ConcreteModel()
model.n = pyo.Param(default=4)
model.x = pyo.Var(pyo.RangeSet(model.n), within=pyo.NonNegativeIntegers)
# model.y = pyo.Var(pyo.RangeSet(model.n), within=pyo.Binary)
# multipliers = [1, 2, 3, 4]

# def objective_func(model):
#     print("Minimizing the: ", sum(i * model.x[i] for i in model.x))
#     return sum(i * model.x[i] for i in multipliers)

def objective_func(model):
    print("Minimizing the: ", pyo.summation(model.x))
    return pyo.summation(model.x)


model.objective = pyo.Objective(rule=objective_func, sense=pyo.minimize)
model.constraint1 = pyo.ConstraintList()

opt = SolverFactory('glpk')
results = opt.solve(model)

for i in range(4):
    print(f"Value for var: x{i+1} is : {pyo.value(model.x[i+1])}")
    # print(f"Value for var: y{i+1} is : {pyo.value(model.y[i+1])}")

Minimizing the:  x[1] + x[2] + x[3] + x[4]
Value for var: x1 is : 0.0
Value for var: x2 is : 0.0
Value for var: x3 is : 0.0
Value for var: x4 is : 0.0


In [6]:
for v in model.component_objects(pyo.Var, active=True):
    print("Variable", v)
    v.pprint()

Variable x
x : Size=4, Index=[1:4]
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      2 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      3 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      4 :     0 :   0.0 :  None : False : False : NonNegativeIntegers


In [7]:
for v in model.component_data_objects(pyo.Var, active=True):
    print("Variable", v)
    print(v.lower, end=", ")
    print(v.value, end=", ")
    print(v.upper, end=", ")
    print(v.fixed, end=", ")
    print(v.stale, end=", ")
    print(v.domain)

Variable x[1]
0, 0.0, None, False, False, NonNegativeIntegers
Variable x[2]
0, 0.0, None, False, False, NonNegativeIntegers
Variable x[3]
0, 0.0, None, False, False, NonNegativeIntegers
Variable x[4]
0, 0.0, None, False, False, NonNegativeIntegers


## Part 2
defineding parameters.

#### Scalar param

In [8]:
model = pyo.ConcreteModel()
model.alpha = pyo.Param(initialize=2.5, within=pyo.NonNegativeReals)
print(pyo.value(model.alpha))

2.5


#### Indexed with a dict

In [9]:
model = pyo.ConcreteModel()
model.i = pyo.RangeSet(1, 4)
model.c = pyo.Param(model.i, initialize={1: 10, 2: 20, 3: 30, 4: 40}, within=pyo.NonNegativeIntegers)
for i in model.i:
    print(pyo.value(model.c[i]), end=", ")

10, 20, 30, 40, 

#### Indexed with a rule (function or lambda)

In [10]:
model = pyo.ConcreteModel()
model.i = pyo.RangeSet(10, 20)

def c_rule(m: pyo.ConcreteModel, i: int) -> float:
    return 2*i

model.c = pyo.Param(model.i, initialize=c_rule)
for i in model.i:
    print(i, "=", pyo.value(model.c[i]), end=", ", sep="")

10=20, 11=22, 12=24, 13=26, 14=28, 15=30, 16=32, 17=34, 18=36, 19=38, 20=40, 

#### Same value for all indices

In [11]:
model = pyo.ConcreteModel()
model.i = pyo.RangeSet(1, 5)
model.w = pyo.Param(model.i, initialize=0.0)
for i in model.i:
    print(i, "=", pyo.value(model.w[i]), end=", ", sep="")

1=0.0, 2=0.0, 3=0.0, 4=0.0, 5=0.0, 

#### Using default when some indices missing

In [12]:
model = pyo.ConcreteModel()
model.i = pyo.RangeSet(1, 5)
model.d = pyo.Param(model.i, initialize={1: 10, 3: 30}, default=0)
for i in model.i:
    print(i, "=", pyo.value(model.d[i]), end=", ", sep="")

1=10, 2=0, 3=30, 4=0, 5=0, 

#### Mutable parameters (change at runtime)

In [13]:
model = pyo.ConcreteModel()
model.I = pyo.RangeSet(1, 3)
model.p = pyo.Param(model.I, initialize=lambda m,i: i*i, mutable=True)

for i in model.I:
    print(i, "=", pyo.value(model.p[i]), end=", ", sep="")
print()
model.p[2].set_value(99)
for i in model.I:
    print(i, "=", pyo.value(model.p[i]), end=", ", sep="")

1=1, 2=4, 3=9, 
1=1, 2=99, 3=9, 

#### Loading from externam data

In [14]:
from pyomo.dataportal import DataPortal

file_path = r"/mnt/Data1/Python_Projects/Pure-Python/P5/06-Pyomo/P2.csv"
model = pyo.ConcreteModel()
model.i = pyo.RangeSet(1, 3)
model.a = pyo.Param(model.i, mutable=True)
data = DataPortal()
data.load(filename=file_path, param=model.a)

model.a.store_values(data.data()["a"])

In [15]:
[pyo.value(model.a[i]) for i in model.a]

[5.0, 6.5, 7.0]

## Part 3
- Variables
    - Decision variable used by the optimizer (unlike Param, which is data).
    - Symbolic in expressions; numeric values appear after solve.
    - Each Var has a domain (type), optional bounds, and optional initial value.

In [16]:
model = pyo.ConcreteModel()
def multipliers_rule(m: pyo.ConcreteModel, i: int) -> float:
    return i * 1.5
model.mul = pyo.Param(tuple(range(1, 11)), initialize=multipliers_rule)
print(f"[INFO] I have these parameters: {list(zip(model.mul.index_set(), [pyo.value(model.mul[i]) for i in model.mul]))}")

print("Now lets defined variables")

[INFO] I have these parameters: [(1, 1.5), (2, 3.0), (3, 4.5), (4, 6.0), (5, 7.5), (6, 9.0), (7, 10.5), (8, 12.0), (9, 13.5), (10, 15.0)]
Now lets defined variables


#### Scalar variable

In [17]:
model = pyo.ConcreteModel()

# Real variable with no bounds
model.x = pyo.Var()

# Nonnegative real with static bounds
model.y = pyo.Var(domain=pyo.NonNegativeReals, bounds=(0, 10))

# Binary and integer
model.z = pyo.Var(domain=pyo.Binary)
model.k = pyo.Var(domain=pyo.Integers, bounds=(0, None))

#### Indexed variables (1-D)

In [18]:
model.i = pyo.RangeSet(1, 4)
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals)

# Access
model.x[1].set_value(2.0)
lb, ub = model.x[2].bounds
print(f"Variable x2 has bounds: {lb}, {ub}")

'pyomo.core.base.var.ScalarVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
Variable x2 has bounds: 0, None


#### Multi-index variables (2-D and higher)

In [ ]:
model.I = pyo.RangeSet(1, 3)
model.J = pyo.Set(initialize=["A", "B"])
model.x = pyo.Var(model.I, model.J, domain=pyo.Reals)

# Access
model.x[2, "B"] = 1.23        # same as set_value(1.23)

model.x[2, "A"].fix(2)
model.x[2, "A"].unfix()

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


#### Bounds (static or rule)

- pyo.Reals (default), pyo.NonNegativeReals, pyo.NonPositiveReals
- pyo.Integers, pyo.NonNegativeIntegers
- pyo.Binary
- pyo.PercentFraction (0..1), pyo.UnitInterval (0..1)


In [20]:
# Static tuple (lb, ub). Use None for ±∞.
model.x = pyo.Var(bounds=(0, None))

# Rule depending on indices and/or Params
def x_bounds(m, i):
    return (0, m.cap[i])   # lb, ub
model.I = pyo.RangeSet(1, 3)
model.cap = pyo.Param(model.I, initialize={1: 5, 2: 7, 3: 9})
model.x = pyo.Var(model.I, bounds=x_bounds)

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.AbstractScalarVar'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.set.FiniteScalarRangeSet'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.set.FiniteScalarRangeSet'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.var.ScalarVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


#### Initialization / starting values

In [21]:
# Scalar
model.x = pyo.Var(initialize=1.0)

# Indexed with dict
model.I = pyo.RangeSet(1, 3)
model.y = pyo.Var(model.I, initialize={1: 0.0, 2: 2.5, 3: 1.0})

# Indexed with rule
def y_init(m, i):
    return 0.5 * i
model.y = pyo.Var(model.I, initialize=y_init)

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.AbstractScalarVar'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.set.FiniteScalarRangeSet'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.set.FiniteScalarRangeSet'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.var.ScalarVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add

#### Fixing, unfixing, and changing bounds at runtime

In [22]:
model.x.fix(3.0)        # x is fixed at 3.0 (becomes a constant for the solver)
model.x.unfix()         # make it a free decision variable again

model.x.setlb(0.0)      # update lower bound
model.x.setub(10.0)     # update upper bound
model.x.set_value(1.5)  # set starting value (does not fix it)

#### Dense vs sparse creation (performance)

In [23]:
# If initialize doesn’t provide values for every index and you don’t want all created:
model.I = pyo.RangeSet(1, 1000)
model.x = pyo.Var(model.I, initialize={}, dense=False)
# Components are created lazily when first referenced; useful for large sparse models.

'pyomo.core.base.set.FiniteScalarRangeSet'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.set.FiniteScalarRangeSet'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.var.ScalarVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


#### Dynamic-sized variable lists (when you can’t predefine an index set)

In [31]:
model.vlist = pyo.VarList(domain=pyo.NonNegativeReals)
# Add variables dynamically
model.vlist.add()
model.vlist.add()       # starting value None
len(model.vlist)        # number of vars added
model.vlist[1]

'pyomo.core.base.var.VarList'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.VarList'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [32]:
model = pyo.ConcreteModel()
model.i = pyo.RangeSet(1, 3)
model.cost = pyo.Param(model.i, initialize={1: 2, 2: 3, 3: 1})

def ub_rule(m: pyo.ConcreteModel, i: int) -> tuple[int, int]: 
    return (0, 5)

model.x = pyo.Var(model.i, domain=pyo.NonNegativeIntegers, bounds=ub_rule,
              initialize=lambda m, i: 0)

model.objective = pyo.Objective(expr=sum(model.cost[i] * model.x[i] for i in model.i), sense=pyo.minimize)
model.constraints = pyo.Constraint(expr=sum(model.x[i] for i in model.i) >= 4)

opt = pyo.SolverFactory("glpk")
opt.solve(model)
print([pyo.value(model.x[i]) for i in model.i], pyo.value(model.objective))

[0.0, 0.0, 4.0] 4.0
